# Predictive Modeling

### Can we Predict Poor Reviews?

**Goal**: Build models to predict which orders will receive poor reviews (<= 3 stars)

**Key Features from [Diagnositc Analysis](04_diagnostic_analytics.ipynb):**
- is_delayed (strongest: r= -0.37)
- delivery_days
- delay_days
- geographic distance
- category urgency
- multi-seller complexity


In [2]:
# Imports, Settings, & DB Connection 
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)

pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)
conn = sqlite3.connect('../data/ecommerce.db')
print("Database connection established")



Database connection established


### 1. Feature Engg & Data Prep

In [ ]:
query = """
SELECT 
    -- Target Variable
    CASE WHEN r.review_score <= 3 THEN 1 ELSE 0 END as is_poor_review,
    
    -- Delivery features (most important)
    JULIANDAY(o.order_delivered_customer_date) - JULIANDAY(o.order_purchase_timestamp) as delivery_days,
    JULIANDAY(o.order_delivered_customer_date) - JULIANDAY(o.order_estimated_delivery_date) as delay_days,
    CASE WHEN JULIANDAY(o.order_delivered_customer_date) > JULIANDAY(o.order_estimated_delivery_date) THEN 1 ELSE 0 END as is_delayed,
    
    -- Order characteristics
    COUNT(DISTINCT oi.product_id) as num_items,
    COUNT(DISTINCT oi.seller_id) as num_sellers,
    SUM(oi.price + oi.freight_value) as total_order_value,
    ROUND(SUM(oi.freight_value) * 100.0 / NULLIF(SUM(oi.price + oi.freight_value), 0), 2) as freight_pct,
    
    -- Geographic Info
    c.customer_state,
    
    -- Primary category
    (
        SELECT COALESCE(pct.product_category_name_english, p2.product_category_name, 'Unknown')
        FROM order_items oi2
        JOIN products p2 ON oi2.product_id = p2.product_id
        LEFT JOIN product_category_name_translation pct ON p2.product_category_name = pct.product_category_name
        WHERE oi2.order_id = o.order_id
        GROUP BY COALESCE(pct.product_category_name_english, p2.product_category_name)
        ORDER BY COUNT(*) DESC
        LIMIT 1
    ) as category,
    
    -- Primary seller state
    (
        SELECT s.seller_state
        FROM order_items oi2
        JOIN sellers s ON oi2.seller_id = s.seller_id
        WHERE oi2.order_id = o.order_id
        GROUP BY s.seller_state
        ORDER BY COUNT(*) DESC
        LIMIT 1
    ) as seller_state
    
FROM orders o
JOIN customers c ON o.customer_id = c.customer_id
JOIN order_items oi ON o.order_id = oi.order_id
LEFT JOIN order_reviews r ON o.order_id = r.order_id
WHERE o.order_status = 'delivered'
  AND o.order_delivered_customer_date IS NOT NULL
  AND r.review_score IS NOT NULL
GROUP BY o.order_id
"""

df = pd.read_sql(query, conn)

print(f"Dataset: {len(df):,} orders")
print(f"Poor review rate: {df['is_poor_review'].mean()*100:.1f}%")
df.head()